# Run eQTL analysis for lupus data

pkill -9 -f joblib

In [4]:
import scanpy as sc
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
                    import itertools
from pybedtools import BedTool
import statsmodels.formula.api as smf
import statsmodels.api as sm
import imp

import os
import pickle as pkl
%matplotlib inline

In [5]:
import sys
sys.path.append('/home/ssm-user/Github/scrna-parameter-estimation/dist/memento-0.0.8-py3.8.egg')
sys.path.append('/home/ssm-user/Github/misc-seq/miscseq/')
import encode
import memento
import run_memento

In [6]:
data_path  = '/data_volume/memento/lupus/'

### Get SNP gene pairs from Gracie's output for sanity checking

In [4]:
# gs_pairs_mine = pd.read_csv(data_path +'mateqtl_input/{}/gene_snp_pairs_{}.csv'.format('eur', '100kb'))

In [5]:
# gracie_nk = pd.read_csv(data_path + 'gracie_eqtls/nk_metasoft_out.txt', sep='\t', skiprows=1, header=None).iloc[:, :3]

In [6]:
# gene_snp_pairs_new = pd.DataFrame()
# gene_snp_pairs_new['gene'] = gracie_nk.iloc[:, 0].str.split('_').str[1]
# gene_snp_pairs_new['rsid'] = gracie_nk.iloc[:, 0].str.split('_').str[0]

In [7]:
# gene_snp_pairs_new.to_csv(data_path + 'gracie_eqtls/gene_snp_pairs.csv', index=False)

### Run memento on full gene-SNP pairs

In [ ]:
# gracie window
imp.reload(run_memento)
for pop in ['asian','eur']:
    print('---', pop, '---')
    run_memento.run_full(pop)

### Combine blocks

In [9]:
for pop in ['asian', 'eur']:
    for ct in ['T4', 'cM', 'ncM', 'T8', 'B', 'NK']:
        result = []
        for block in range(5):
            result.append(pd.read_csv('/data_volume/memento/lupus/memento_full/100kb/{}_{}_block_{}.csv'.format(pop, ct, block)))
        result = pd.concat(result)
        result['FDR'] = memento.util._fdrcorrect(result['de_pval'])
        result['statistic'] = result['de_coef']/result['de_se']
        result = result[['tx', 'gene', 'statistic', 'de_pval', 'FDR','de_coef']]
        result.columns = ["SNP","gene","t-stat","p-value","FDR","beta"]
        print(result.query('FDR < 0.1').shape[0])
        result.to_csv(data_path + 'full_analysis/memento/100kb/{}_{}.csv'.format(pop, ct), index=False)

48863
93693
35330
53100
35401
20379
61242
93087
35766
32052
29423
22323


### Select genes from pseudobulk for Matrix eQTL

In [2]:
cts = ['T4', 'cM', 'ncM', 'T8', 'B', 'NK']

In [7]:
for pop in ['asian', 'eur']:
    for ct in cts:
        print(pop, ct)
        results = pd.read_csv(data_path + 'full_analysis/memento/100kb/{}_{}.csv'.format(pop, ct))

        pseudobulk = pd.read_csv(data_path + 'pseudobulk/{}_{}.csv'.format(pop, ct), index_col=0, sep='\t')
        pseudobulk = pseudobulk.loc[:, results['gene'].drop_duplicates().tolist()]
        pseudobulk.T.to_csv(data_path + 'full_analysis/mateqtl/pseudobulk/{}_{}.csv'.format(pop, ct), sep='\t')

asian T4
asian cM
asian ncM
asian T8
asian B
asian NK
eur T4
eur cM
eur ncM
eur T8
eur B
eur NK
